# Model testing notebook

In this notebook we will test the model to see how it performs on new data.

We will download proper and non-proper tracks from YouTube and score them using the model.

We will then compare the scores to the actual labels and calculate the accuracy of the model.

In [ ]:
# Import necessary libraries
import os
import pickle  # Assuming the model is saved using joblib
import librosa  # Assuming you are using librosa to process audio tracks
import numpy as np

# Load the trained model
model_path = 'src/backend/resources/your_model_file.pkl'  # Replace with your actual model file name
model = pickle.load(open(model_path, 'rb'))


def extract_features(file_path):
    y, sr = librosa.load(file_path)

    # Extract features
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0)
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr).T, axis=0)
    zero_crossings = np.mean(librosa.feature.zero_crossing_rate(y))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    rms_energy = np.mean(librosa.feature.rms(y=y))

    # Combine all features into a single array
    features = np.hstack([mfccs, chroma, spectral_contrast, zero_crossings, spectral_bandwidth, rms_energy])
    return features

# Get predictions for tracks
tracks_dir = 'src/backend/tracks/'
for track_file in os.listdir(tracks_dir):
    track_path = os.path.join(tracks_dir, track_file)
    features = extract_features(track_path)
    prediction = model.predict([features])
    print(f"Track: {track_file}, Prediction: {prediction}")